In [1]:
import os
import asyncio
from dotenv import load_dotenv
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict

from agents import Agent, Runner, trace, function_tool
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content


In [2]:
load_dotenv()

True

### Part 1: Agent workflow

In [6]:
instructions1 = "You are a sales agent working for DAnhI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for DAnhI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for DAnhI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini"
)

In [ ]:
message = 'Write a cold sales email'

# Run the agent
result = Runner.run_streamed(sales_agent1, message)

# Stream the response
async for event in result.stream_events():
    if event.type == 'raw_response_event' and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end='', flush=True)

In [ ]:
# Sales Picker Agent: pick the best email MANUALLY
sales_picker = Agent(
    name="Sales Picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
with trace('Selection from sale emails'):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    
    outputs = [result.final_output for result in results]
    
    emails = 'Cold sales email:\n\n'.join(outputs)
    
    best_email = await Runner.run(sales_picker, emails)
    
    print(f'Best sale email:\n{best_email.final_output}')

### Part 2: Tools

In [44]:
@function_tool
def send_email(body: str):
    '''Send email with the given body to all sales prospects using SendGrid API'''
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email('anh.leduy04@hcmut.edu.vn') # Verified sender
    to_email = To('duyanhlucas302@gmail.com') # To recipient
    subject = 'Cold sales email'
    content = Content('text/plain', body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {'status': 'success'}

In [45]:
# Automatically convert function to tool, with the boilerplate json created
send_email

FunctionTool(name='send_email', description='Send email with the given body to all sales prospects using SendGrid API', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10bcc3880>, strict_json_schema=True)

In [46]:
# We can also convert an agent into a tool
# If this tool is called, it will run the agent to make a call to LLM
tool1 = sales_agent1.as_tool(
    tool_name='sales_agent1',
    tool_description=message
) 

In [47]:
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10bcc3eb0>, strict_json_schema=True)

In [48]:
tool2 = sales_agent2.as_tool(tool_name='sales_agent2', tool_description=message)
tool3 = sales_agent3.as_tool(tool_name='sales_agent3', tool_description=message)

tools = [tool1, tool2, tool3, send_email]

In [49]:
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10bcc3eb0>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10bcc3250>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent

In [51]:
# Sales Manager Agent:
manager_instructions = """You are a sales manager working for DAnhI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
**Note**: Only pick ONE BEST email and use the send_email tool to send that ONE BEST email to the user.
"""

sales_manager_agent = Agent(
    name='Sales Manager',
    instructions=manager_instructions,
    tools=tools,
    model='gpt-4o-mini'
)

with trace('Sales Manager'):
    result = await Runner.run(sales_manager_agent, 'Send a cold sales email address to "Dear CEO"')
    print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to "Dear CEO." If you need anything else, let me know!
- 9 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Part 3: Handoff

In both cases, an Agent can collaborate with another Agent

But there are difference between Handoff vs. Agent as tool:
- Handoff: Passing its entire job to another agent (when passing the job, it will not return)
- Agent as tool: Use another agent as a tool to complete its job (receive a response from another agent and continue its job)

In [ ]:
# Define subject writer agent:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

subject_writer = Agent(
    name="Email subject writer",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)

subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a subject for a cold sales email"
)

# Define html converter agent:
html_converter = Agent(
    name="HTML email body converter",
    instructions=html_instructions,
    model="gpt-4o-mini"
)

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a text email body to an HTML email body"
)


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("anh.leduy04@hcmut.edu.vn")
    to_email = To("duyanhlucas302@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [56]:
tools = [subject_tool, html_tool, send_html_email]
handoffs = [emailer_agent]

In [ ]:
# Define email sender agent (handoff task from Sales Manager)
emailer_instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name="Email Manager",
    instructions=emailer_instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

In [ ]:
sales_manager_instructions = "You are a sales manager working for DAnhI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You ONLY select the ONE BEST EMAIL using your own judgement of which email will be most effective. \
After picking the email, you MUST HANDOFF to the Email Manager agent to format and send the email."

sales_manager_agent_with_handoff = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager_agent_with_handoff, message)